In [2]:
import random
import string

#### De forma a agilizar o regresso às aulas pós pandemia, a ESTG decidiu reorganizar a distribuição de horários. Ajude a escola neste processo, ao desenvolver uma solução baseada em AGs q seja capaz de realizar um escalonamento de tds as atividades, c base nas seguintes regras:
#### - 1. As aulas só podem ser realizadas em salas q estejam livres pra um determinado espaço temporal
#### - 2. Cd docente só pode lecionar uma aula de cd vez
#### - 3. As salas devem ser suficientemente grds pra acomodar uma turma (se possível o tam das mms deve ser adequado o + possível ao tam da turma).
#### Pra desenv uma solução pra este problema considere a existência de Salas, Docentes, UCs, Turmas, Bloco de Horas (considere blocos de 2h), Aulas (combinação de turma, UC, docente, bloco de horas, e sala), e horário.
##### a) Defina e crie o seu universo de dados, considerando 𝑛 entidades q representem Salas, Docentes, UCs, Turmas, Blocos de Horas, Aulas, e Horário.

In [1]:
# Salas
salas = [
    {"id": 1, "nome": "Sala A", "capacidade": 30},
    {"id": 2, "nome": "Sala B", "capacidade": 40},
    {"id": 3, "nome": "Sala C", "capacidade": 25}
]

# Docentes
docentes = [
    {"id": 1, "nome": "Prof. Ana"},
    {"id": 2, "nome": "Prof. Bruno"},
    {"id": 3, "nome": "Prof. Carla"}
]

# Unidades Curriculares
ucs = [
    {"id": 1, "nome": "IA"},
    {"id": 2, "nome": "ED"},
    {"id": 3, "nome": "Sistemas Distribuídos"}
]

# Turmas
turmas = [
    {"id": 1, "nome": "LEI1", "tamanho": 28},
    {"id": 2, "nome": "LEI2", "tamanho": 35}
]

# Blocos de Horas (2h cada)
blocos_horas = [
    {"id": 1, "dia": "Segunda", "periodo": "Manhã"},
    {"id": 2, "dia": "Segunda", "periodo": "Tarde"},
    {"id": 3, "dia": "Quarta",  "periodo": "Manhã"},
    {"id": 4, "dia": "Quarta",  "periodo": "Tarde"}
]

# Aula = combinação de turma + UC + docente + bloco + sala
# (vamos gerar depois nos indivíduos, mas aqui estão as entidades base)

##### b) Construa uma população composta por 100 indivíduos cujos cromossomas são compostos por combinações randoms de aulas q fazem parte de um horário.

In [3]:
# Gerar uma aula aleatória
def gerar_aula():
    return {
        "turma": random.choice(turmas),
        "uc": random.choice(ucs),
        "docente": random.choice(docentes),
        "sala": random.choice(salas),
        "bloco": random.choice(blocos_horas)
    }

# Criar um cromossoma (horário com n aulas por turma)
def criar_individuo(aulas_por_turma=3):
    individuo = []
    for turma in turmas:
        for _ in range(aulas_por_turma):
            aula = gerar_aula()
            aula["turma"] = turma  # garantir que respeita a turma atual
            individuo.append(aula)
    return individuo

# Criar população de 100 indivíduos
def criar_populacao(tamanho_populacao=100, aulas_por_turma=3):
    return [criar_individuo(aulas_por_turma) for _ in range(tamanho_populacao)]

In [4]:
# 📌 Exemplo de uso
populacao = criar_populacao()

# Mostrar 1º horário
print("Exemplo de cromossoma (horário):")
for aula in populacao[0]:
    print(f"Turma {aula['turma']['nome']} | UC: {aula['uc']['nome']} | Docente: {aula['docente']['nome']} | Sala: {aula['sala']['nome']} | Bloco: {aula['bloco']['dia']} - {aula['bloco']['periodo']}")

Exemplo de cromossoma (horário):
Turma LEI1 | UC: Sistemas Distribuídos | Docente: Prof. Carla | Sala: Sala A | Bloco: Quarta - Manhã
Turma LEI1 | UC: Sistemas Distribuídos | Docente: Prof. Carla | Sala: Sala C | Bloco: Segunda - Tarde
Turma LEI1 | UC: Sistemas Distribuídos | Docente: Prof. Carla | Sala: Sala C | Bloco: Segunda - Manhã
Turma LEI2 | UC: Sistemas Distribuídos | Docente: Prof. Ana | Sala: Sala B | Bloco: Segunda - Manhã
Turma LEI2 | UC: ED | Docente: Prof. Ana | Sala: Sala A | Bloco: Segunda - Tarde
Turma LEI2 | UC: IA | Docente: Prof. Bruno | Sala: Sala B | Bloco: Segunda - Manhã


##### c)  Identifique qual seria a função de fitness + adequada pra esta solução. Implemente-a

In [5]:
def calcular_fitness(individuo):
    fitness = 100  # começa com pontuação máxima

    ocupacao_salas = {}
    ocupacao_docentes = {}

    for aula in individuo:
        bloco = aula["bloco"]["id"]
        sala = aula["sala"]["id"]
        docente = aula["docente"]["id"]
        turma = aula["turma"]
        sala_capacidade = aula["sala"]["capacidade"]
        turma_tamanho = turma["tamanho"]

        # Verificar conflito de sala no mesmo bloco
        chave_sala = (bloco, sala)
        if chave_sala in ocupacao_salas:
            fitness -= 10  # penalização por conflito de sala
        else:
            ocupacao_salas[chave_sala] = True

        # Verificar conflito de docente no mesmo bloco
        chave_docente = (bloco, docente)
        if chave_docente in ocupacao_docentes:
            fitness -= 10  # penalização por conflito de docente
        else:
            ocupacao_docentes[chave_docente] = True

        # Verificar adequação da sala
        if sala_capacidade < turma_tamanho:
            fitness -= 5  # penalização por não caber
        else:
            diferenca = sala_capacidade - turma_tamanho
            fitness -= diferenca * 0.2  # penalização leve por "espaço a mais"

    return max(fitness, 0)  # fitness nunca deve ser negativo

In [6]:
# 📌 Exemplo de uso com a população da alínea b):
# Calcular fitness para todos os indivíduos
fitness_pop = [(ind, calcular_fitness(ind)) for ind in populacao]

# Ordenar por melhor fitness
fitness_pop.sort(key=lambda x: x[1], reverse=True)

# Mostrar os 5 melhores horários
print("\nTop 5 horários:")
for i in range(5):
    print(f"{i+1}. Fitness: {fitness_pop[i][1]}")


Top 5 horários:
1. Fitness: 87.79999999999998
2. Fitness: 85.79999999999998
3. Fitness: 83.79999999999998
4. Fitness: 83.79999999999998
5. Fitness: 82.6


##### d) Calcule o valor de fitness pra cd individuo q está presente na população inicial.

In [7]:
# Avaliar fitness de toda a população
fitness_populacao = [(individuo, calcular_fitness(individuo)) for individuo in populacao]

# Ordenar por fitness decrescente
fitness_populacao.sort(key=lambda x: x[1], reverse=True)

# Mostrar os 5 melhores horários
print("\nTop 5 indivíduos com melhor fitness:")
for i, (ind, fit) in enumerate(fitness_populacao[:5]):
    print(f"{i+1}. Fitness: {fit}")
    for aula in ind:
        print(f"   Turma: {aula['turma']['nome']}, UC: {aula['uc']['nome']}, Docente: {aula['docente']['nome']}, Sala: {aula['sala']['nome']}, Bloco: {aula['bloco']['dia']} {aula['bloco']['periodo']}")
    print("-" * 60)


Top 5 indivíduos com melhor fitness:
1. Fitness: 87.79999999999998
   Turma: LEI1, UC: IA, Docente: Prof. Bruno, Sala: Sala A, Bloco: Quarta Manhã
   Turma: LEI1, UC: Sistemas Distribuídos, Docente: Prof. Ana, Sala: Sala A, Bloco: Segunda Manhã
   Turma: LEI1, UC: Sistemas Distribuídos, Docente: Prof. Ana, Sala: Sala A, Bloco: Segunda Tarde
   Turma: LEI2, UC: IA, Docente: Prof. Ana, Sala: Sala C, Bloco: Quarta Manhã
   Turma: LEI2, UC: ED, Docente: Prof. Carla, Sala: Sala C, Bloco: Quarta Tarde
   Turma: LEI2, UC: Sistemas Distribuídos, Docente: Prof. Bruno, Sala: Sala B, Bloco: Segunda Tarde
------------------------------------------------------------
2. Fitness: 85.79999999999998
   Turma: LEI1, UC: Sistemas Distribuídos, Docente: Prof. Ana, Sala: Sala B, Bloco: Quarta Manhã
   Turma: LEI1, UC: IA, Docente: Prof. Ana, Sala: Sala A, Bloco: Quarta Tarde
   Turma: LEI1, UC: IA, Docente: Prof. Carla, Sala: Sala A, Bloco: Segunda Tarde
   Turma: LEI2, UC: IA, Docente: Prof. Carla, Sala: